In [1]:
import ipywidgets as widgets
print(f'IPywidgets version: {widgets.__version__}')

import numpy as np
print(f'NumPy version: {np.__version__}')

import matplotlib.pyplot as plt
import matplotlib
print(f'Matplotlib version: {matplotlib.__version__}')

from ipywidgets import interact, interactive, fixed, interact_manual

import pandas as pd
print(f'Pandas version: {pd.__version__}')

IPywidgets version: 7.5.1
NumPy version: 1.18.5
Matplotlib version: 3.2.2
Pandas version: 1.0.5


Problema 1. (2 puncte) Definiti o functie polinomiala de gradul 3, $f:\mathbb{R} \rightarrow \mathbb{R}$, cu coeficienti constanti prestabiliti. Aplicati algoritmul gradient descent pentru a vedea cum evolueaza cautarea minimului. Folositi minim doua controale ipywidgets: unul pentru pozitia initiala a lui $x$, altul pentru coeficientul $\alpha>0$ cu care se inmulteste gradientul. Gradientul va fi calculat analitic de voi sau folosind biblioteca [autograd](https://github.com/HIPS/autograd). 
Modificarea facuta prin metoda gradient descent este:
$$
x = x - \alpha \cdot f'(x)
$$
Se vor efectua minim 10 iteratii (optional: numarul de iteratii poate fi dat printr-un control ipywidgets), se vor marca pe grafic pozitiile succesive, in mod convenabil. 

In [2]:
import numpy as np
import matplotlib.pyplot as plt

a, b, c, d = 1, -1, -1, 1

def func(x: float) -> float:
    """
    Calculeaza functia polinomiala intr-un punct.
    
    :param x: punctul x in care se calculeaza functia
    
    :return: rezultatul functiei in punctul x
    """
    return a * np.power(x, 3) + b * np.power(x, 2) + c * x + d

def gradient(x: float) -> float:
    """
    Calculeaza gradientul intr-un punct.
    
    :param x: punctul x in care se calculeaza gradientul
    
    :return: rezultatul gradientului in punctul x
    """
    return 3 * a * np.power(x, 2)  + 2 * b * x + c

def gradient_descent(x0: float = 1.5, alfa: float = 0.1, num_it: int = 10) -> None:
    """
    Calculeaza gradientul descendent pentru functia x si afiseaza pe graficul functiei punctele rezultate.
    
    :param x0: punctul x initial
    :param alfa: rata de invatare
    :param num_it: numarul de iteratii care se vor executa
    """
    x_range: np.array =  np.arange(-10, 10, 0.01)
    fx: np.array = func(x_range)
    
    fig, ax = plt.subplots(1, figsize = (10, 10))
    
    ax.scatter(x0, func(x0))
    for _ in range(num_it):
        x0 -= alfa * gradient(x0)
        ax.scatter(x0, func(x0))

    ax.plot(x_range, fx, 'b-')
    ax.grid()
    ax.axis([-5, 5, -100, 100])
    plt.show()

interact(gradient_descent, x0 = (-1.5, 3., 0.05), alfa = (0.01, 0.3, 0.01), num_it = (10, 15));

interactive(children=(FloatSlider(value=1.5, description='x0', max=3.0, min=-1.5, step=0.05), FloatSlider(valu…

Problema 2. (3 puncte) Generati o lista de $n=100$ de perechi de valori $\{x_i, y_i\}_{i=0,n-1}$ in intervalul [-20, 10), afisati aceste valori pe un grafic, impreuna cu o dreapta definita de o functie liniara $y=a \cdot x+b$. Intr-un alt plot afisati, ca histograma, distanta dintre punctele de coordonate $(x_i, y_i)$ si punctele de intersectie ale verticalelor duse prin $x_i$ cu dreapta data, $\hat{y}_i$. Dreapta trebuie sa fie controlabila din widgets, prin cei doi coeficienti $a$ si $b$. Constatati modificarea histogramei in functie de pozitia dreptei si afisati mean squared error: $$MSE=\frac{1}{n} \cdot \sum_{i=0}^{n-1} (y_i - (a\cdot x_i + b))^2$$.

*Indicatii:*
1. Pentru generare de valori distribuite uniform in intervalul [0, 1) puteti folosi functia [numy.random.uniform](https://docs.scipy.org/doc/numpy/reference/generated/numpy.random.uniform.html) si sa faceti inmultire si adunare in mod convenabil.
1. Puteti opta sa returnati cele $n$ puncte sub forma `vector_x`, `vector_y`.

In [3]:
np.random.seed(99)

def generate_pairs(left: int = -20, right: int = 10, n:int = 100) -> (np.array, np.array):
    """
    Genereza o lista de n perechi de valori (x,y) in intervalul [left, right)
    
    :param left: capatul din stanga al intervalului
    :param right: capatul din dreapta al intervalului
    :param n: numarul de perechi generate
    
    :return: liste cu coordonatele x, y ale perechilor
    """
    pairs = np.random.uniform(left, right, size = (2, n))  
    return pairs[0,:], pairs[1,:]

def f(x: float, a: float, b: float) -> float:
    """
    Calculeaza valoarea functiei in punctul x, functia liniara avand coeficientii a si b
    
    :param x: punctul in care se calculeaza functia
    :param a: coeficientul lui x
    :param b: termenul liber
    
    :return: valoarea functiei in punctul x
    """
    return a * x + b

def distance_between_points(vector_x: np.array, vector_y: np.array, a: float, b: float) -> np.array:
    """
    Calculeaza distanta dintre punctele de coordonate  (𝑥𝑖,𝑦𝑖) si punctele de intersectie ale verticalelor 
    duse prin  𝑥𝑖  cu dreapta data,  𝑦̂𝑖.
    
    :param vector_x: coordonatele x ale punctelor
    :param vector_y: coordonatele y ale punctelor
    :param a: coeficientul lui x al functiei f
    :param b: coeficientul termenului liber al functiei f
    
    :return: array care contine distantele
    """
    
    distance = lambda x, y  : np.sqrt((y - f(x, a, b)) ** 2) # sqrt((x-x)^2 + (y-f(x))^2) = sqrt((y-f(x))^2)
    return np.array([distance(random_x, random_y) for random_x, random_y in zip(vector_x, vector_y)])
    
def plot_pairs(vector_x: np.array, vector_y: np.array, a: float = 1, b: float = 2) -> None:
    """
    Deseneaza doua grafice:
        1) graficul care contine punctele generate random si dreapta data de functia f
        2) graficul care contine histograma cu distantele dintre punctele date si punctele de intersectie 
           ale verticalelor duse prin  𝑥𝑖  cu dreapta data
        
    :param vector_x: coordonatele x ale punctelor
    :param vector_y: coordonatele y ale punctelor
    :param a: coeficientul lui x al functiei f
    :param b: coeficientul termenului liber al functiei f
    """
    fx = f(vector_x, a, b)
    fig, (ax1, ax2) = plt.subplots(2, figsize = (10, 15))
    
    ax1.plot(vector_x, vector_y, 'm.') 
    ax1.plot(vector_x, fx, 'b-') 
    ax1.axis([-25, 15, -25, 15])
    ax1.set(xlabel='x', ylabel='y')
    
    distances: np.array = distance_between_points(vector_x, vector_y, a, b)

    ax2.hist(distances, bins = 50, density = True)
    ax2.set(ylabel='distanta')

    plt.show()
    
    
vector_x, vector_y = generate_pairs()
interact(plot_pairs, vector_x = fixed(vector_x), vector_y = fixed(vector_y), a = (-20, 10), b = (-20, 10));

interactive(children=(IntSlider(value=1, description='a', max=10, min=-20), IntSlider(value=2, description='b'…

Problema 3. (2 puncte) Incarcati fisierul `iris.data` din [setul de date iris](https://archive.ics.uci.edu/ml/datasets/iris). In functie de alegerile exprimate de un utilizator, afisati intr-un grafic 2D coloanele numerice alese (de exemplu, coloana 0 si coloana 2). Numele coloanelor se afla in fisierul iris.names.

In [4]:
header_list: list = ["sepal length", "sepal width", "petal length", "petal width", "class"]
data_frame = pd.read_csv('iris.data', header=None, usecols=[0, 1, 2, 3, 4], skipinitialspace=True, names = header_list)

groups = data_frame.groupby('class')

input1 = widgets.Dropdown(
    options=["sepal length", "sepal width", "petal length", "petal width"],
    value = "sepal length",
    description='Option 1'
)
input2 = widgets.Dropdown(
    options=["sepal length", "sepal width", "petal length", "petal width"],
    value = "sepal width",
    description='Option 2'
)

def plot_function(input1: str, input2: str) -> None:
    """
    Deseneaza un grafic avand pe axa Ox input1 si pe axa Oy input2, input-urile reprezentand 
    nume de coloane din data_frame.
    
    :param input1: optiunea pentru axa Ox
    :param input2: optiunea pentru axa Oy
    """
    plt.figure(figsize=(10, 5))
    for name, group in groups:
        plt.plot(group[input1], group[input2], 'o', label = name)
#         plt.hist(group[input1], label=name + ' ' + input1)
#         plt.hist(group[input2], label=name + ' ' + input2)
        plt.xlabel(input1)
        plt.ylabel(input2)
    plt.legend()
    plt.show()

widgets.interact(plot_function, input1 = input1, input2 = input2);

interactive(children=(Dropdown(description='Option 1', options=('sepal length', 'sepal width', 'petal length',…

Problema 4 (3 puncte) Generati $n$ perechi de puncte aleatoare, folosind o functie $f:\mathbb{R} \rightarrow \mathbb{R}$ de alease e voi (de exemplu: functie polinomiala + zgomot aleator adaugat). Alegeti 5 metode de interpolare din [scipy.interpolate](https://docs.scipy.org/doc/scipy/reference/interpolate.html) si reprezentati grafic valorile interpolate. Folositi controale ipywidgets cel putin pentru alegerea lui $n$ si a metodei de interpolare aleasa.

In [13]:
import ipywidgets as widgets
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import krogh_interpolate, pchip_interpolate, NearestNDInterpolator, interpolate, barycentric_interpolate

np.random.seed(10)
a, b, c, d = 1, -1, -1, 1

def func(x: float) -> float:
    """
    Calculeaza functia polinomiala intr-un punct.
    
    :param x: punctul x in care se calculeaza functia
    
    :return: rezultatul functiei in punctul x
    """
    return a * np.power(x, 3) + b * np.power(x, 2) + c * x + d

def generate_points(n: int = 10) -> (np.array, np.array):
    """
    Genereaza n perechi de puncte aleatoare folosind functia func si le adauga zgomot.
    
    :param n: numarul de perechi generate
    
    :return: array cu coordonatele x ale punctelor, array cu coordonatele y ale punctelor
    """
    x: np.array = np.linspace(0.0, 5.0, n)
    y: np.array = func(x) + np.random.normal(0, 3, len(x))
    return x, y

def plot_krogh_interpolate(x_observed: np.array, y_observed: np.array) -> None:
    """
    Aplica metoda de interpolare krogh pentru punctele date ca parametru, apoi reprezinta grafic rezultatul.
    
    :param x_observed: coordonatele x ale punctelor
    :param y_observed: coordonatele y ale punctelor
    """
    x: np.array = np.linspace(min(x_observed), max(x_observed), num=100)
    y: np.array = krogh_interpolate(x_observed, y_observed, x)
    plt.plot(x_observed, y_observed, "o", label="observation")
    plt.plot(x, y, label="krogh interpolation")
    plt.legend()
     
def plot_pchip_interpolate(x_observed: np.array, y_observed: np.array) -> None:
    """
    Aplica metoda de interpolare pchip pentru punctele date ca parametru, apoi reprezinta grafic rezultatul.
    
    :param x_observed: coordonatele x ale punctelor
    :param y_observed: coordonatele y ale punctelor
    """
    x: np.array = np.linspace(min(x_observed), max(x_observed), num=100)
    y: np.array = pchip_interpolate(x_observed, y_observed, x)
    plt.plot(x_observed, y_observed, "o", label="observation")
    plt.plot(x, y, label="pchip interpolation")
    plt.legend()
    
def plot_NearestNDInterpolator(x: np.array, y: np.array) -> None:
    """
    Aplica metoda de interpolare NearestNDInterpolator pentru punctele date ca parametru, apoi reprezinta grafic rezultatul.
    
    :param x: coordonatele x ale punctelor
    :param y: coordonatele y ale punctelor
    """
    z: np.array = np.hypot(x, y)
    X: np.array = np.linspace(min(x), max(x))
    Y: np.array = np.linspace(min(y), max(y))
    X, Y = np.meshgrid(X, Y)  # 2D grid for interpolation
    interp = NearestNDInterpolator(list(zip(x, y)), z)
    Z = interp(X, Y)
    plt.pcolormesh(X, Y, Z, shading='auto')
    plt.plot(x, y, "ok", label="input point")
    plt.legend()
    plt.colorbar()
    plt.axis("equal")
    
def plot_interp2d(x: np.array, y: np.array) -> None:
    """
    Aplica metoda de interpolare interp2d pentru punctele date ca parametru, apoi reprezinta grafic rezultatul.
    
    :param x: coordonatele x ale punctelor
    :param y: coordonatele y ale punctelor
    """    
    xx, yy = np.meshgrid(x, y)
    z: np.array = np.sin(xx**2+yy**2)
    f = interpolate.interp2d(x, y, z, kind='cubic')
    xnew = np.arange(-5.01, 5.01, 1e-2)
    ynew = np.arange(-5.01, 5.01, 1e-2)
    znew = f(xnew, ynew)
    plt.plot(x, z[0, :], 'ro-', xnew, znew[0, :], 'b-')
    
def plot_barycentric_interpolator(x_observed: np.array, y_observed: np.array) -> None:
    """
    Aplica metoda de interpolare barycentric_interpolator pentru punctele date ca parametru,
    apoi reprezinta grafic rezultatul.
    
    :param x_observed: coordonatele x ale punctelor
    :param y_observed: coordonatele y ale punctelor
    """  
    x: np.array = np.linspace(min(x_observed), max(x_observed), num=100)
    y: np.array = barycentric_interpolate(x_observed, y_observed, x)
    plt.plot(x_observed, y_observed, "o", label="observation")
    plt.plot(x, y, label="barycentric interpolation")
    plt.legend()


drop_down = widgets.Dropdown(options=['krogh interpolation','pchip interpolation',
                                      'Piecewise linear interpolant in N dimensions', 'interp2d',
                                      'barycentric interpolation'],
                                description='Choose',
                                disabled=False)

slider = widgets.IntSlider(value=5, min=4, max=15, step=1, description='n:', disabled=False, orientation='horizontal')

def plot_interpolation(option: str, n: int) -> None:
    """
    Apeleaza functia de interpolare care corespunde cu optiunea aleasa pentru n puncte.
    
    :param option: metoda de interpolare care trebuie apelata
    :param n: numarul de puncte generate aleator
    """
    x_observed, y_observed = generate_points(n)

    plt.figure(figsize=(13, 5))
    
    if option == 'krogh interpolation':
        plot_krogh_interpolate(x_observed, y_observed)
    elif option == 'pchip interpolation':
        plot_pchip_interpolate(x_observed, y_observed)
    elif option == 'Piecewise linear interpolant in N dimensions':
        plot_NearestNDInterpolator(x_observed, y_observed)
    elif option == 'interp2d':
        plot_interp2d(x_observed, y_observed)
    elif option == 'barycentric interpolation':
        plot_barycentric_interpolator(x_observed, y_observed)
    plt.show()        
        
widgets.interact(plot_interpolation, option = drop_down, n = slider);

interactive(children=(Dropdown(description='Choose', options=('krogh interpolation', 'pchip interpolation', 'P…